In [1]:
print("hello")

hello


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
KEY=os.getenv("OPENAI_API_KEY")


# Load url

In [8]:
from langchain_community.document_loaders import UnstructuredURLLoader

loaders=UnstructuredURLLoader(urls=[
    "https://www.icicibank.com/personal-banking/cards/credit-card",
    "https://www.hdfcbank.com/personal/pay/cards/credit-cards",
])

data=loaders.load()

In [9]:
len(data)

2

# Split Documents

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)

docs=text_splitter.split_documents(data)

In [13]:
len(docs)

123

# Embeddings

In [17]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


In [19]:
from langchain_community.vectorstores import FAISS

In [20]:
vector_index=FAISS.from_documents(docs,embeddings)


In [26]:
# While save model as pickle file we may get some errors
# https://stackoverflow.com/questions/77605224/cannot-pickle-thread-rlock-object-while-serializing-faiss-object
vector_index.save_local("credicard")

# Load Model

In [37]:
vectordb=FAISS.load_local("credicard",OpenAIEmbeddings(),allow_dangerous_deserialization=True)
retrieverdb=vectordb.as_retriever()

In [29]:
from langchain.chains import RetrievalQAWithSourcesChain

In [38]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.9,max_tokens=400)
chain=RetrievalQAWithSourcesChain.from_chain_type(OpenAI(temperature=0), chain_type="stuff", retriever=retrieverdb)
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_types={}, partial_variables={}, template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: Which state/country\'s law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.\nSource: 28-pl\nContent: No Waiver. Failure or de

In [41]:
response=chain({
    "question": "Benifits of Freedom Credit Card?"
},return_only_outputs=True)

In [42]:
response

{'answer': ' The benefits of the Freedom Credit Card include lifetime free credit card and vouchers worth up to INR 5,000, 5X cashpoints on EMI spends, and access to exclusive offers and discounts. \n',
 'sources': 'https://www.hdfcbank.com/personal/pay/cards/credit-cards'}

In [44]:
print(response['answer'])

 The benefits of the Freedom Credit Card include lifetime free credit card and vouchers worth up to INR 5,000, 5X cashpoints on EMI spends, and access to exclusive offers and discounts. 



 The benefits of the Freedom Credit Card include lifetime free credit card and vouchers worth up to INR 5,000, 5X cashpoints on EMI spends, and access to exclusive offers and discounts. 
